In [2]:
import duckdb

con = duckdb.connect()

# Load landing data
con.execute("""
    CREATE TABLE customer_landing AS
    SELECT * FROM read_json_auto('/Users/arunkumarthirunavukarasu/IdeaProjects/nd027-Data-Engineering-Data-Lakes-AWS-Exercises/project/starter/customer/landing/*.json');
""")

con.execute("""
    CREATE TABLE accelerometer_landing AS
    SELECT * FROM read_json_auto('/Users/arunkumarthirunavukarasu/IdeaProjects/nd027-Data-Engineering-Data-Lakes-AWS-Exercises/project/starter/accelerometer/landing/*.json');
""")

con.execute("""
    CREATE TABLE step_trainer_landing AS
    SELECT * FROM read_json_auto('/Users/arunkumarthirunavukarasu/IdeaProjects/nd027-Data-Engineering-Data-Lakes-AWS-Exercises/project/starter/step_trainer/landing/*.json');
""")

In [76]:
con.execute("SELECT COUNT(*) FROM customer_landing").fetchall()


[(956,)]

In [77]:
con.execute("SELECT COUNT(*) FROM step_trainer_landing").fetchall()


[(28680,)]

In [78]:
con.execute("SELECT COUNT(*) FROM accelerometer_landing").fetchall()


[(81273,)]

In [68]:
con.execute("""DROP TABLE IF EXISTS accelerometer_trusted;

CREATE TABLE accelerometer_trusted AS
WITH trusted_customers AS (
    SELECT *
    FROM customer_landing
    WHERE shareWithResearchAsOfDate IS NOT NULL
),
joined AS (
    SELECT
        a.user,
        a.timestamp,
        a.x,
        a.y,
        a.z
    FROM accelerometer_landing a
    INNER JOIN trusted_customers c
        ON a.user = c.email
)
SELECT *
FROM joined;""").fetchall()


[(40981,)]

In [79]:
con.execute("SELECT count(*) FROM accelerometer_trusted").fetchall()


[(40981,)]

In [58]:
con.execute("""
  DROP TABLE IF EXISTS customer_trusted;

CREATE TABLE customer_trusted AS
SELECT
    *
FROM customer_landing
WHERE shareWithResearchAsOfDate IS NOT NULL;""")

In [80]:
con.execute("SELECT COUNT(*) FROM customer_trusted").fetchall()


[(482,)]

In [71]:
con.execute("""
  DROP TABLE IF EXISTS customers_curated;

CREATE TABLE customers_curated AS
WITH joined AS (
    SELECT
        c.*
    FROM customer_trusted c
    INNER JOIN accelerometer_trusted a
        ON c.email = a.user
)
SELECT DISTINCT *
FROM joined;""")

In [81]:
con.execute("SELECT COUNT(*) FROM customers_curated").fetchall()


[(482,)]

In [82]:
con.execute("""
DROP TABLE IF EXISTS step_trainer_trusted;

CREATE TABLE step_trainer_trusted AS
SELECT
    s.sensorReadingTime,
    s.serialNumber,
    s.distanceFromObject
FROM step_trainer_landing s
INNER JOIN customers_curated c
    ON s.serialNumber = c.serialNumber;""")

In [83]:
con.execute("SELECT COUNT(*) FROM step_trainer_trusted").fetchall()


[(14460,)]

In [74]:
con.execute("""
DROP TABLE IF EXISTS machine_learning_curated;

CREATE TABLE machine_learning_curated AS
WITH
-- Filter accelerometer_trusted to curated customers
accelerometer_filtered AS (
    SELECT
        a.user,
        a.timestamp,
        a.x,
        a.y,
        a.z
    FROM accelerometer_trusted a
    INNER JOIN customers_curated c
        ON a.user = c.email
),

-- Filter step_trainer_trusted to curated customers
step_filtered AS (
    SELECT
        s.sensorReadingTime,
        s.serialNumber,
        s.distanceFromObject
    FROM step_trainer_trusted s
    INNER JOIN customers_curated c
        ON s.serialNumber = c.serialNumber
),

-- Join step trainer + accelerometer on timestamp
joined AS (
    SELECT
        s.sensorReadingTime,
        s.distanceFromObject,
        a.user,
        a.timestamp,
        a.x,
        a.y,
        a.z
    FROM step_filtered s
    INNER JOIN accelerometer_filtered a
        ON s.sensorReadingTime = a.timestamp
)

-- Final ML curated table
SELECT *
FROM joined;""")

In [75]:
con.execute("SELECT COUNT(*) FROM machine_learning_curated").fetchall()


[(43681,)]